<a href="https://colab.research.google.com/github/Svanzi/Jazz-Improvisation/blob/main/Jazz_Improv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import re
import numpy as np
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from tensorflow.keras.utils import to_categorical
# import play
import os

In [ ]:
!git clone https://github.com/Svanzi/Jazz-Improvisation.git
%cd Jazz-Improvisation

Cloning into 'Jazz-Improvisation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 49 (delta 12), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 43.07 KiB | 7.18 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/Jazz-Improvisation/Jazz-Improvisation/Jazz-Improvisation


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from music21 import converter, instrument, note, chord, stream

In [ ]:
songs = glob("MIDI/*.mid")
# Per verificare che il caricamento dei file sia andato a buon fine e che i file midi siano presenti
print(songs)

['MIDI/Jacob_Allen_-_Test.mid', 'MIDI/michel_petrucciani_caravan_solo_-_michel_petrucciani_caravan_solo.mid', 'MIDI/Andre_and_Schwandt_-_Dream_a_Little_Dream.mid', 'MIDI/Ensemble-_Isaac_Piano_Week_4_with_structure__-_Ensamble__Geo_Drum_110_Bpm.mid', 'MIDI/Moab_Berckmans_de_Oliveira_-_MBOM06.mid', 'MIDI/Nintendo_-_Dr._Mario.mid', 'MIDI/Cameron_Lee_Simpson_-_Illumination_Nights.mid']


## Load all the notes in an array

In [ ]:
def get_notes():
  notes = []
  for file in songs:
    midi = converter.parse(file)
    notes_to_parse = []

    try:
      parts = instrument.partitionByInstrument(midi)
    except:
      pass
    if parts: # file has instrument parts
      notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes

    for element in  notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

  path = '/content/drive/My Drive/University/AI2Arts/data'
  os.makedirs(path, exist_ok=True)

  with open(os.path.join(path, 'notes'), 'wb') as filepath:
    pickle.dump(notes, filepath)

  return notes

## Define the input and output sequences

In [ ]:
# n_vocab represent the total number of unique notes in the variable 'notes_data'
def sequences(notes, n_vocab):
  sequence_length = 100

  pitchnames = sorted(set(item for item in notes))

  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

  network_input = []
  network_output = []

  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i: i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

  n_patterns = len(network_input)

  network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

  network_input = network_input / float(n_vocab)

  network_output = to_categorical(network_output)

  return (network_input, network_output)

## Create the Network

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Flatten

def create_network(network_input, n_vocab):
  """Create the model architecture"""
  model = Sequential()
  model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512, return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(256))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

  return model

## Train

In [ ]:
from keras.callbacks import ModelCheckpoint

def train(model, network_input, network_output, epochs):
  """Train the neural network"""

  #filepath = "weights.best.music3.epoch-{epoch:02d}.keras"
  filepath = "/content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras"

  checkpoint = ModelCheckpoint(
      filepath,
      monitor='loss',
      verbose=2,
      save_best_only=True,
      mode='min'
  )

  model.fit(network_input, network_output, epochs=epochs, batch_size=64, callbacks = [checkpoint])

In [ ]:
def train_network():
  """
  Get notes
  Generates input and putput sequences
  Create model
  TRains the model for the given epochs
  """

  epochs = 200

  notes_data = get_notes()
  print('Notes processed')

  n_vocab = len(set(notes_data))
  print('Vocab generated')

  network_in, network_out = sequences(notes_data, n_vocab)
  print('Input and Output processed')

  model = create_network(network_in, n_vocab)
  print('Model created')
  #return model
  print('Training in progress')
  train(model,network_in, network_out, epochs)
  print('Training completed')

In [ ]:
notes_data = get_notes()
print(f"Extracted {len(notes_data)} notes/chords.")
print("First 10 entries:", notes_data[:10])

Extracted 3133 notes/chords.
First 10 entries: ['A4', 'C4', 'C#4', 'G#3', 'A3', 'F3', 'F#3', 'G#4', 'A4', 'C#4']


In [ ]:
train_network()

Notes processed
Vocab generated
Input and Output processed
Model created
Training in progress
Epoch 1/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 4.8591
Epoch 1: loss improved from inf to 4.70176, saving model to /content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 173s 3s/step - loss: 4.8559
Epoch 2/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 4.5129
Epoch 2: loss improved from 4.70176 to 4.50198, saving model to /content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 165s 3s/step - loss: 4.5127
Epoch 3/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 4.3900
Epoch 3: loss improved from 4.50198 to 4.37567, saving model to /content/drive/MyDrive/University/AI2Arts/weights.best.music3.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 203s 3s/step - loss: 4.3897
Epoch 4/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 4.2439
Epoch 4: loss improved from 4.37567 to 4.25782, saving model to /content/drive/MyDrive/Un

KeyboardInterrupt: 

## Note Generation


In [ ]:
def get_inputSequence(notes, pitchnames, n_vocab):
  """ Prepare the sequence used by the Neural Network """
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

  sequence_length = 100
  network_input = []
  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i: i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])

  network_input = np.reshape(network_input, (len(network_input), 100, 1))

  return (network_input)

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
  """ Generate notes from the neural network based on a sequence of notes """

  start = np.random.randint(0, len(network_input)-1)

  int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

  pattern = list(network_input[start])
  prediction_output = []

  print('Generating notes....')

  for note_indec in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern.append(index)
    pettern = pattern[1:len(pattern)]

  print('Notes Generated...')
  return prediction_output

In [ ]:
def create_midi(prediction_output):
  """ Convert the output from the preduction to notes and create a midi file from the notes """
  offset = 0
  output_notes = []

  for pattern in prediction_output:
    if('.' in pattern) or pattern.isdigit():
      notes_in_chord = pattern.split('.')
      notes = []
      for current_note in notes_in_chord:
        new_note = note.Note(int(current_note))
        new:note.storeInstrument = instrument.Guitar()
      new_chord = chord.Chord(notes)
      new_chord.offset = offset
      output_notes.append(new_note)
      offset += 0.5

    midi_stream = stream.Stream(output_notes)

    print('Saving Output file as midi...')
    midi_stream.write('midi', fp='test_output.mid')

In [ ]:
def generate():
  """Generate a piano midi file"""
  path = '/content/drive/My Drive/University/AI2Arts/data'
  with open(os.path.join(path, 'notes'), 'rb') as filepath:
    notes = pickle.lead(filepath)

  pitchnames = sorted(set(item for item in notes))
  n_vocab = len(set(notes))

  print('Initating music generation process...')

  network_input = get_inputSequence(notes, pitchnames, n_vocab)
  normalized_input = network_input / float(n_vocab)
  model = create_network(normalized_input, n_vocab)
  model.summary()
  print('Loading model weights')
  model.load_weights('weights.best.music3.keras')
  print('Model Loaded')
  prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
  create_midi(prediction_output)

In [ ]:
generate()

## Play MIDI

In [ ]:
import pygame

def play_music(music_file):
  """ Stream music with mixer.music module in blocking manner.
  This will stream the sound from disk while playing """

  clock = pygame.time.clock()
  try:
    pygame.mixer.music.load(music_file)
    print("Music file %s loaded!" % music_file)
  except pygame.error:
    print("File %s not found! (%s)"% (music_file, pygame.get_error()))
    return
  pygame.mixer.music.play()
  while pygame.mixer.music.get_busy():
    clock.tick(30)

In [ ]:
def play_midi(midi_file):
  freq = 44100
  bitsize = -16
  channels = 2
  buffer = 1024
  pygame.init(freq, bitsize, channels, buffer)

  pygame.mixer.music.set_volume(0.8)

  try:
    play_music(midi_file)
  except KeyboardInterrupt:
    pygame.mixer.music.fadeout(1000)
    pygame.mixer.music.stop()
    raise SystemExit

In [ ]:
play_midi('test_output.mid')